In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries
import alpaca_trade_api as tradeapi
from config import *
import os
import time

# Compute Stock Beta

In [2]:
#SET ALAPACA API ENVIRONMENT VARIABLES 

HEADERS={'APCA-API-KEY-ID':API_KEY,'APCA-API-SECRET-KEY':SECRET_KEY}
alpaca=tradeapi.REST(API_KEY,SECRET_KEY,BASE_URL,api_version='v2')

In [3]:
def get_bars(symbol,interval='15Min',start = '2021-01-01'):
    bar=alpaca.get_bars(symbol,interval,start=start).df
    return bar

In [4]:
#SET ALPHA VINTAGE ENVIRONMENT VARIABLES 
ts=TimeSeries(key=ALPHA_API_KEY,output_format='pandas')
dfy = get_bars('SPY','1D')
dfx1=get_bars('ADBE','1D')
dfx2=get_bars('INTC','1D')

In [5]:
dfy.tail()

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2022-06-27 04:00:00+00:00,391.12,391.3600,387.44,388.59,65967268,552197,389.085434
2022-06-28 04:00:00+00:00,390.30,393.1600,380.53,380.65,86376255,641197,384.912075
2022-06-29 04:00:00+00:00,381.16,382.2700,378.42,380.34,64478427,519970,380.393953
2022-06-30 04:00:00+00:00,376.27,380.6582,372.56,377.25,112269284,799487,377.083807
2022-07-01 04:00:00+00:00,376.60,381.7000,373.80,381.24,74165514,601221,378.221516


In [6]:
import statsmodels.api as sm
from statsmodels import regression 

def get_stock_beta(dfx1,dfx2,dfy):
    dfy_returns=dfy[['close']].pct_change()[1:].values
    dfx1_returns=dfx1[['close']].pct_change()[1:].values
    dfx2_returns=dfx2[['close']].pct_change()[1:].values
    
    def linreg(X1,X2,Y):
        X1=sm.add_constant(X1)
        X2=sm.add_constant(X2)
        model1=regression.linear_model.OLS(Y,X1).fit()
        model2=regression.linear_model.OLS(Y,X2).fit()
        return model1.params[1],model2.params[1]
    return linreg(dfx1_returns,dfx2_returns,dfy_returns)

In [7]:
x1_beta,x2_beta=get_stock_beta(dfx1,dfx2,dfy)

In [8]:
x1_beta,x2_beta

(0.3695190007406155, 0.3434310230309465)

In [9]:
alpaca.get_latest_trade('ADBE')

TradeV2({   'c': ['@'],
    'i': 4790,
    'p': 368.46,
    's': 100,
    't': '2022-07-01T19:59:59.601443072Z',
    'x': 'V',
    'z': 'C'})

In [10]:
x1_price=alpaca.get_latest_trade('ADBE').p
x2_price=alpaca.get_latest_trade('INTC').p
long_limit=int(3000/x1_price)
short_limit=int((long_limit*x1_price*x1_beta)/(x2_beta*x2_price))
bought=0
sold=0

In [11]:
def place_order(pair,bought,sold,window1=5,window2=60):
    S1 = get_bars(pair[0],'1Min')['close']
    S2 = get_bars(pair[1],'1Min')['close']
    dt_stocks=pd.merge(S1,S2,on='timestamp')
    dt_stocks.columns=pair
    # If window length is 0, algorithm doesn't make sense, so exit
    if (window1 == 0) or (window2 == 0):
        return 0
    # Compute rolling mean and rolling standard deviation
    ratios = dt_stocks[pair[0]]/dt_stocks[pair[1]]
    ma1 = ratios.rolling(window=window1,
                               center=False).mean()[-1]
    ma2 = ratios.rolling(window=window2,
                               center=False).mean()[-1]
    std = ratios.rolling(window=window2,
                        center=False).std()[-1]
    zscore = (ma1 - ma2)/std    
    
    Account=alpaca.get_account()

    r='No Acction'
    status=zscore
    
    #Sell short Buy long if the z-score is >1
    if zscore>1 and sold<short_limit and bought<long_limit:
        side='sell'
        r=alpaca.submit_order(pair[1],7,side,'market','gtc')
        k=alpaca.submit_order(pair[0],1,'buy','market','gtc')
        Account=alpaca.get_account()
        status=r.side +" "+"7"+" "+pair[1]+" at $"+str(alpaca.get_trades(pair[1])[0].p) +" Acct Val:"+Account.portfolio_value
        sold=sold+7
        bought=bought+1
    #Buy long if the z-score is <1
    elif zscore<-1 and bought<long_limit and sold<short_limit :
        side='buy'
        r=alpaca.submit_order(pair[0],1,side,'market','gtc')
        k=alpaca.submit_order(pair[1],7,'sell','market','gtc')
        Account=alpaca.get_account()
        status=r.side +" "+"1"+" "+pair[0]+" at $"+str(alpaca.get_trades(pair[0])[0].p) +" Acct Val:"+Account.portfolio_value
        bought=bought+1
        sold=sold+7
    #Clear positions if the z-score between -0.5 and 0.5
    elif abs(zscore)<0.5:
        alpaca.close_all_positions()
        Account=alpaca.get_account()
        sold=0
        bought=0
        status='No Transaction'+ " [INFO] Acct Val: $ " + Account.portfolio_value +" Z-score " + str(zscore) 
    return status,bought,sold


In [12]:
while(True):
    status, bought, sold=place_order(['ADBE','INTC'],bought,sold,5,60)
    print(status)
    time.sleep(300)

sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/INTC/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/INTC/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/INTC/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/INTC/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/INTC/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/INTC/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/INTC/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/INTC/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/INTC/bars 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/INTC/bars 3 more time(s)...


IndexError: list index out of range